In [47]:
#encoding = utf8
import jieba
from gensim.models.keyedvectors import KeyedVectors

def load_word2vec(vec_path = '/home/gwang3/workspace/chatbot/chinese_word2vec/cn.skipgram.bin'):
    return KeyedVectors.load_word2vec_format('/home/gwang3/workspace/chatbot/chinese_word2vec/cn.skipgram.bin', binary=True, unicode_errors='ignore')

def load_core_words(core_word_path = 'elec_core_word'):
    core_word_list = []
    infile = open(core_word_path)
    for line in infile:
        core_word_list.append(line.strip())
    return core_word_list

def cal_sen_vec(sentence, core_word_list, model):
    seg_list = jieba.cut(sentence)
    #for item in seg_list:
    #    print item
    vec = []
    for item in core_word_list:
        max_simi = 0
        for sen_seg in seg_list:
            simi_tmp = 0
            try:
                simi_tmp = model.similairity(item, sen_seg)
            except:
                simi_tmp = 0
            if simi_tmp > max_simi:
                max_simi = simi_tmp
        vec.append(max_simi)
    return vec


#Core QA Dic Format:
#core_qa_dic[question][0] = question_vector
#core_qa_dic[question][1] = answer
def load_core_qa(core_qa_path = 'elec_core_qa', core_word_list, model):
    core_qa_dic = {}
    infile = open(core_qa_path)
    for line in infile:
        line = line.strip().split('|')
        question = line[0].strip()
        answer = linep[1].strip()
        core_qa_dic[question] = []
        core_qa_dic[question].append(cal_sen_vec(question, core_word_list, model))
        core_qa_dic[question].append(answer)
    return core_qa_dic
    

def cal_sen_simi(sentence1, sentence2):
    return 0
    
def main():
    model = load_word2vec()
    core_word_list = load_core_words()
    core_qa_dic = load_core_qa(core_word_list = core_word_list, model = model)
    
    
    

SyntaxError: invalid syntax (<ipython-input-47-4414a35e7332>, line 53)

In [7]:
cal_sen_vec("今天天气不错，我准备去吃个包子。包子是大肉馅的，特别特别香！")

今天天气
不错
，
我
准备
去
吃
个
包子
。
包子
是
大
肉馅
的
，
特别
特别
香
！


In [18]:
from gensim.models.keyedvectors import KeyedVectors
model = KeyedVectors.load_word2vec_format('/home/gwang3/workspace/chatbot/chinese_word2vec/cn.skipgram.bin', binary=True, unicode_errors='ignore')

In [46]:
print model.similarity(u'电费', u'水费')

0.71662710525
